<a href="https://colab.research.google.com/github/Pankhuri1999/Artificial_Intelligence_Assignment-3/blob/main/Question-3/Assignment3_Ques3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The steps that I followed are - 


1) In the first step, which is that of object detection, I did it through color recognition method. I detected the the object by assigning yellow colour numpy array

2) In the second step, I created the code for Kalman filter

3) In third steps, I created the code which computes mesaured, calculated and actal distance

In [1]:
 # In the first step, which is that of oebject detection, I did it through color recognition method. 
 #I detected the the object by assigning yellow colour numpy array
 
import cv2
import numpy as np


class Detector:
    def __init__(self):
        self.low = np.array([22, 60, 200])
        self.high = np.array([60, 255, 255])

    def detect(self, frame):
        hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Create masks with color ranges
        mask = cv2.inRange(hsv_img, self.low, self.high)

        # Find Contours
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)

        box = (0, 0, 0, 0)
        for cnt in contours:
            (x, y, w, h) = cv2.boundingRect(cnt)
            box = (x, y, x + w, y + h)
            break

        return box

In [4]:
# Creation for kalman filter code
import numpy as np


class KalmanFilter:
    kf = cv2.KalmanFilter(4, 2)
    kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)

    U = [0, 0]
    #X = np.zeros(shape=(4,1))
    X = np.array([[910], [277], [0], [0]])  # U1 = 231, 860, 323, 960
    X1 = np.array([[57], [251], [0], [0]])   # U2 = 197,   0, 306, 114
    std=[0.1,0,0] 
    U = np.array(U).reshape(2,-1) 
    dt = 1
        # error in measurements x and y (ie, std deviation of measurements)
    xm_std,ym_std,std_acc = std[0],std[1],std[2]

        # Define the State Transition Matrix A
    A = np.array([[1, 0, dt, 0],
                            [0, 1, 0, dt],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]])
        # input control matrix
    B = np.array([[(dt**2)/2, 0],
                            [0, (dt**2)/2],
                            [dt,0],
                            [0,dt]])

        # since we are tracking only position of a moving object we have (we are not tracking velocity)
    H = np.array([[1,0,0,0],[0,1,0,0]])

        #process covariance matric  # for now we initialize as an identity matrix
    P = np.eye(A.shape[0])

        #process noise covariance matrix  // Dynamic noise
        #standard deviation of position as the standard deviation of acceleration multiplied by dt**2/2
    Q = np.array([[(dt**4)/4, 0, (dt**3)/2, 0],
                            [0, (dt**4)/4, 0, (dt**3)/2],
                            [(dt**3)/2, 0, dt**2, 0],
                            [0, (dt**3)/2, 0, dt**2]]) * std_acc**2

        #measurement noise covariance matrix  // Measurement noise
    R = np.array([[xm_std**2,0],
                           [0, ym_std**2]])

    process_noise = 0
    measurement_noise = 0

    def predict(self):
        self.X = np.dot(self.A,self.X) + np.dot(self.B,self.U) + self.process_noise
        self.P = np.dot(np.dot(self.A,self.P),self.A.T) + self.Q
        return self.X[0:2]

    def update(self,Xm):
        Xm = np.array(Xm).reshape(2,1)
        
        # calculate kalaman gain
        # K = P * H'* inv(H*P*H'+R)
        denominator = np.dot(self.H,np.dot(self.P,self.H.T)) + self.R
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(denominator)) #shape: (4,2)
        

        # measurments
        C = np.eye(Xm.shape[0])
        Xm = np.dot(C,Xm) + self.measurement_noise

        # update the predicted_state to get final prediction of iteration and process_cov_matrix
        self.X = self.X + np.dot(K,(Xm - np.dot(self.H,self.X)))

        #update process cov matrix
        self.P = (np.eye(K.shape[0]) - np.dot(np.dot(K,self.H),self.P))
        return self.X[0:2]


In [ ]:
# Creation of actual, measured and predicted distnace
import cv2


cap = cv2.VideoCapture("ball.mp4")

# Load detector
od = Detector()

# Load Kalman filter to predict the trajectory
kf = KalmanFilter()
 

while True:
    ret, frame = cap.read()
    if ret is False:
        break

    bbox = od.detect(frame)
    x, y, x2, y2 = bbox
    cx = int((x + x2) / 2)
    cy = int((y + y2) / 2)

    predicted = kf.predict()
    (x1, y1) = kf.update([cx, cy])
    #cv2.rectangle(frame, (x, y), (x2, y2), (255, 0, 0), 4)
    cv2.circle(frame, (cx, cy), 2, (0, 0, 255), 10)
    cv2.circle(frame, (predicted[0], predicted[1]), 20, (255, 0, 0), 4)
    cv2.circle(frame, (x1, y1), 20, (255, 255, 255), 4)

    cv2.putText(frame, "Actual Position", (cx+15, cy+15), 0, 0.5, (0, 0, 255), 2)
    cv2.putText(frame, "Predicted Position", (predicted[0]-20, predicted[1]-20), 0, 0.5, (255, 0, 0), 2)
    cv2.putText(frame, "Measured Position", (x1-40, y1-40), 0, 0.5, (255, 255, 255), 2)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(150)
    if key == 27:
        break